# ipmort 

In [1]:
import os
import numpy as np
import pandas as pd

import random
import torch

from torch.utils.data import Dataset
import torch.nn as nn
import torch
from torch import optim
from torch.utils.data import DataLoader

os.environ["CUDA_VISIBLE_DEVICES"]="3"

# utils

In [2]:
def preprocess(image_list):
    """ Normalize Image and Permute (N,H,W,C) to (N,C,H,W)
    Args:
      image_list: List of images (9000, 32, 32, 3)
    Returns:
      image_list: List of images (9000, 3, 32, 32)
    """
    image_list = np.array(image_list)
    image_list = np.transpose(image_list, (0, 3, 1, 2))
    image_list = (image_list / 255.0) * 2 - 1
    image_list = image_list.astype(np.float32)
    return image_list

In [3]:

def count_parameters(model, only_trainable=False):
    if only_trainable:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())

def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    np.random.seed(seed)  # Numpy module.
    random.seed(seed)  # Python random module.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


In [24]:
def save_model(model, file_name):
    file_path = os.path.join('..', 'checkpoints', file_name)
    torch.save(model.state_dict(), file_path)

# data

## check data

In [ ]:
trainX = np.load(os.path.join('..', 'datasets', 'trainX.npy'))
trainX.shape


In [ ]:
valX = np.load(os.path.join('..', 'datasets', 'valX.npy'))
valX.shape


In [ ]:
valY = np.load(os.path.join('..', 'datasets', 'valY.npy'))
valY.shape


## dataset

In [4]:
class ImageDataset(Dataset):
    def __init__(self, file_name):
        self.file_name = file_name
        self.setup()
        
    def setup(self):
        self.data = np.load(os.path.join('..', 'datasets', self.file_name))
        self.data = preprocess(self.data)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        return image

In [ ]:
imageDataset = ImageDataset('trainX.npy')

In [ ]:
imageDataset[10].shape, len(imageDataset)

## dataloader

In [ ]:
train_loader = DataLoader(imageDataset, batch_size=64, shuffle=True)


In [ ]:
a = next(iter(train_loader))
a.shape

# model

In [5]:
class AE(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2)
        )
 
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 5, stride=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 9, stride=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 17, stride=1),
            nn.Tanh()
        )

    def forward(self, x):
        code = self.encoder(x)
        x_hat  = self.decoder(code)
        return code, x_hat

In [ ]:
aeModel = AE()
print(aeModel)

# train

## data

In [ ]:
imageDataset = ImageDataset('trainX.npy')
train_loader = DataLoader(imageDataset, batch_size=64, shuffle=True)


## build model

In [6]:
def build_model(device):
    aeModel = AE()
    aeModel = aeModel.to(device) #!
    return aeModel

In [ ]:
aeModel = build_model(device)
print(aeModel)

## train on batch

In [26]:
def train_model(model, optimizer, train_loader, loss_function, device):
    total_loss = 0.0
    model.train()
    
    for i, data in enumerate(train_loader):

        x = data.to(device)
        code, x_hat = model(x)
        
        batch_loss = loss_function(x_hat, x)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

        total_loss += batch_loss.item()
    
#     avg_batch_loss = total_loss
    avg_batch_loss = total_loss / len(train_loader) #? len(train_loader) = 幾個batch
    
    return model, optimizer, avg_batch_loss
    

In [20]:
# loss = nn.MSELoss()
# input = torch.randn(2, 1, requires_grad=True)
# target = torch.randn(2, 1)
# output = loss(input, target)
# output.backward()
# input, target, output

In [8]:
imageDataset = ImageDataset('trainX.npy')
train_loader = DataLoader(imageDataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

aeModel = build_model(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(aeModel.parameters(), lr=1e-5, weight_decay=1e-5)

aeModel, optimizer, train_loss = train_model(aeModel, optimizer, train_loader, loss_function, device) 


In [9]:
train_loss

0.0035203646596740274

## train on epoch

In [28]:
imageDataset = ImageDataset('trainX.npy')
train_loader = DataLoader(imageDataset, batch_size=64, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

aeModel = build_model(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(aeModel.parameters(), lr=1e-5, weight_decay=1e-5)

for epoch in range(100):
    aeModel, optimizer, train_loss = train_model(aeModel, optimizer, train_loader, loss_function, device) 
    print(epoch, train_loss)
    
    if epoch % 10 == 0:
        file_name = 'epoch_%s_loss_%s.pth'%(epoch, train_loss)
        save_model(aeModel, file_name)
    

0 0.2251900650728914
1 0.18107222155072636
2 0.1514452630303856
3 0.13176027399704868
4 0.1190653052881248
5 0.11064204775301137
6 0.10469059371634533
7 0.10029113522373644
8 0.09690998158508674
9 0.09395781282643627
10 0.09148211569938444
11 0.0893798822298982
12 0.08749928303006896
13 0.08575173388970525
14 0.08415552368737701
15 0.08272958356410937
16 0.08148219755717687
17 0.08014767673006631
18 0.07897871973595225
19 0.07783780022895426
20 0.07673409306689312
21 0.07571928875338763
22 0.07464096527126499
23 0.07368508782377817
24 0.0728012708512912
25 0.07197398370257894
26 0.07114040389432943
27 0.07033306127764229
28 0.06965883580365576
29 0.06891632220126633
30 0.06822566660051059
31 0.06760457323345923
32 0.06705864527283754
33 0.0664275569519154
34 0.06592858902045659
35 0.06535841958751355
36 0.06486303143595394
37 0.0643361922549574
38 0.0638407010837157
39 0.06334892713597842
40 0.0629552748698489
41 0.06250425204074472
42 0.062121382921485974
43 0.06168840419417037
44 0.0